In [1]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install numpy
%pip install -U scikit-learn
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached matplotlib-3.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:010m
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn import tree
from matplotlib import pyplot as plt


start = 8
target = 5
nodes = [(5, 8), (8, 5), (5, 12), (8, 12)]
failure = 12000
iterations = 15000
data = []
for i in range(0, failure):
    data.append(np.loadtxt(f'traffic/{i}.txt'))
data = np.array(data)


data_test = []
for i in range(failure, iterations):
    data_test.append(np.loadtxt(f'traffic/{i}.txt'))
data_test = np.array(data_test)

In [7]:
def predict_next_n_samples(model, n, sample):
    predictions = np.zeros(n)
    predictions[0] = model.predict(np.array(sample).reshape(1,-1))
    for i in range(1, n):
        predictions[i] = model.predict([[predictions[i-1]]])
    return predictions

def predict_multiple(model, y_test, window, samples):
    predictions = np.array([])
    for i in range(0, samples, window):
        prediction  = predict_next_n_samples(model, window, y_test[i])
        predictions = np.concatenate((prediction, predictions), axis=None)
    return predictions

def prediction_graph(y_test, y_predicted, title):
    x_axis = range(len(y_test))
    plt.title(f'{title}')
    plt.plot(x_axis, y_test, label='real', linewidth=1)
    plt.plot(x_axis, y_predicted, label='predicted', linewidth=1)
    plt.legend()
    plt.show()

In [ ]:
regressors = [
    RandomForestRegressor(n_estimators = 10, max_depth=2, random_state=0),
    RandomForestRegressor(n_estimators = 50, max_depth=10, random_state=0),
    RandomForestRegressor(n_estimators = 100, max_depth=20, random_state=0),
    KNeighborsRegressor(n_neighbors=3, algorithm='auto'),
    KNeighborsRegressor(n_neighbors=6, algorithm='ball_tree'),
    KNeighborsRegressor(n_neighbors=12, algorithm='kd_tree'),
    LinearRegression(),
    LinearRegression(fit_intercept=False),
    MLPRegressor(hidden_layer_sizes=(100,50,20)),
    MLPRegressor(hidden_layer_sizes=(100,50)),
    MLPRegressor(hidden_layer_sizes=(50,20)),
    SVR(C=1.0, epsilon=0.2),
    DecisionTreeRegressor(max_depth=5)
]

samples = 3000
for start, target in nodes:
    print(f'{start} -> {target}')
    # dane treningowe
    y_train = np.zeros(failure-1)
    X_train = np.zeros(failure-1)
    for i in range(0, failure-1):
        X_train[i] = data[i][start, target]
        y_train[i] = data[i+1][start, target]
    X_train = [i.flatten() for i in X_train ]

    # dane testowe
    X_test = np.zeros(iterations - failure)
    for i, traffic in enumerate(data_test):
        X_test[i] = traffic[start, target]
    y_test = X_test[1:samples]
    
    for model in regressors:
        print(model.__class__.__name__)
        model.fit(X_train, y_train)
        predictions = predict_multiple(model, X_test, 5, samples)
        mape = mean_absolute_percentage_error(y_test[50:], predictions[50:-1])
        print(f'MAPE: {np.round(mape, 3)}')
#         prediction_graph(y_test[:], predictions[:-1], f'{start} -> {target}   {model.__class__.__name__}')  

5 -> 8
RandomForestRegressor
MAPE: 2.238
RandomForestRegressor
MAPE: 2.383
RandomForestRegressor
MAPE: 2.375
KNeighborsRegressor
MAPE: 2.391
KNeighborsRegressor
MAPE: 2.377
KNeighborsRegressor
MAPE: 2.383
LinearRegression
MAPE: 2.383
LinearRegression
MAPE: 2.382
MLPRegressor
MAPE: 2.396
MLPRegressor
MAPE: 2.389
MLPRegressor
MAPE: 2.385
SVR
MAPE: 2.375
DecisionTreeRegressor
MAPE: 2.382
8 -> 5
RandomForestRegressor
MAPE: 2.014
RandomForestRegressor
MAPE: 2.251
RandomForestRegressor
MAPE: 2.223
KNeighborsRegressor
MAPE: 2.208
KNeighborsRegressor
MAPE: 2.267
KNeighborsRegressor
MAPE: 2.26
LinearRegression
MAPE: 2.254
LinearRegression
MAPE: 2.245
MLPRegressor
MAPE: 2.269
MLPRegressor
MAPE: 2.247
MLPRegressor
MAPE: 2.274
SVR
MAPE: 2.122
DecisionTreeRegressor
MAPE: 2.197
5 -> 12
RandomForestRegressor
MAPE: 3.148
RandomForestRegressor
MAPE: 3.559
RandomForestRegressor
MAPE: 3.458
KNeighborsRegressor
MAPE: 4.199
KNeighborsRegressor
MAPE: 3.454
KNeighborsRegressor
MAPE: 3.355
LinearRegression
MA